#### Ref docs

#### Import libs

In [1]:
import pandas as pd
import re
import numpy as np

#### Source file path

In [ ]:
# Set path to folder holding two files downloaded from NHSD
# Zip file download: https://files.digital.nhs.uk/05/D0E0F1/GPWPracticeCSV.112022.zip
# File 1: 39. General Practice – November 2022 Individual Level.csv                     (data)
# File 2: General Practice Practice-Level CSV. Overall Definitions.xlsx saved as CSV    (meta)
path_to_nov22_folder = ''   ## user sets path to unzipped downloaded folder after generating meta CSV. 
                            
if path_to_nov22_folder[-1] = '/':
    path_to_nov22_folder = path_to_nov22_folder[:-1]  #remove trailing / in unix-style path if set above

#### Read in files

In [2]:
nov_22_meta = pd.read_csv(
    f"{path_to_nov22_folder}/General Practice Practice-Level CSV. Overall Definitions.csv",
    dtype={
        "Columns": "string",
        "Name of Field": "string",
        "Description": "string",
        "Comments": "string",
    },
)

nov_22_data = pd.read_csv(
    f"{path_to_nov22_folder}/39. General Practice – November 2022 Practice Level.csv"
)


C:\Users\quent\AppData\Local\Temp\ipykernel_20492\47386914.py:11: DtypeWarning: Columns (631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720) have mixed types. Specify dtype option on import or set low_memory=False.
  nov_22_data = pd.read_csv(


In [3]:
nov_22_data.shape # (6429, 721)
nov_22_meta.shape # (721, 4)

(721, 4)

In [4]:
staff_groups = {
    "^.*?_GP_.*$":"GPs",
    "^.*_NURSE_*$":"Nurses",
    "^.*_DPC_*$":"Direct Patient Care",
    "^.*_ADMIN_*$":"Admin/Non-clinical",
}

#### Filter dataset to columns for desired final splits

In [5]:
## Options (easier) are:
## Sex, Staff Role (contains Staff Group), Unit (FTE | Headcount)
## Could be possible to add in age group for headcount but not FTE

pat = re.compile(r'(?:^(MALE|FEMALE)_(GP|NURSE|ADMIN)_(.*?)_(HC|FTE)$)|PRAC_CODE')
keep_cols = list(filter(pat.search, nov_22_data.columns))
df = nov_22_data[keep_cols]

In [6]:
#### Define some variables to use in transformations (id_var, name_var)
#### Pivot longer to allow for easy derivation of job role, staff group
id_var = 'PRAC_CODE'
name_var = 'Name of Field'
rename_dict = {id_var: 'location_id', 
               name_var: '_'.join(name_var.lower().split(' '))
              }
df = df.melt(id_vars = id_var, 
             value_vars = [i for i in keep_cols if i !=id_var], 
             var_name = name_var
            )

#### Derive splits from unpivoted headers

In [7]:
df[["sex", "staff_group", "staff_role", "unit"]] = df[name_var].str.extract(
    r"(MALE|FEMALE)_(GP|NURSE|ADMIN)_(.*?)_(HC|FTE)"
)

#### Add in description field (Optional)

In [8]:
# Optional join on meta info to bring in field description
df = df.merge(nov_22_meta[[name_var, "Description"]], on=name_var, how="left")

#### Clean headers

In [9]:
# This intentionally doesn't lower case the optional description field as would later be dropped
df.rename(rename_dict, inplace=True, axis=1)

#### Handling invalid data in 'value' column

From _comments_ column in meta:

'ND' denotes practices who have No (valid) Data in the current extraction (excluding TIS data), having not passed data quality validation rules, or where their submitted records are staff who have left or not yet joined. For practices who provided 0 records overall for this staff group in the current extraction, the cell will be populated by 0. These practices will form part of the Sub-ICB Location level estimates for their area. These practices will form part of the Sub-ICB Location level estimates for their area/

example: df['value'][df['value']=='ND']

In [10]:
# Use np.NAN as substitution in case sum over staff groups to get totals later (versus uses -9999 etc)
df['value'].replace(to_replace = 'ND', value = np.NaN, inplace=True)

#### Convert 'value' column to float

In [11]:
df['value'] = df['value'].astype('float')

#### Optional inspect basic output dataset info

In [12]:
# name_var (e.g. name_of_field) will be dropped later but retained now for info purposes
# description field left in just for info
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 642900 entries, 0 to 642899
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   location_id    642900 non-null  object 
 1   name_of_field  642900 non-null  object 
 2   value          641264 non-null  float64
 3   sex            642900 non-null  object 
 4   staff_group    642900 non-null  object 
 5   staff_role     642900 non-null  object 
 6   unit           642900 non-null  object 
 7   Description    642900 non-null  string 
dtypes: float64(1), object(6), string(1)
memory usage: 44.1+ MB


#### Look at totals

In [68]:
pat_staff_group_totals = re.compile(r'^TOTAL_(?:GP|NURSES|DPC|ADMIN)_(?:FTE|HC)$|{0}'.format(id_var))
keep_cols_staff_group_totals = list(filter(pat_staff_group_totals.search, nov_22_data.columns))
df_totals = nov_22_data[keep_cols_staff_group_totals]
df_totals

,PRAC_CODE,TOTAL_GP_HC,TOTAL_GP_FTE,TOTAL_NURSES_HC,TOTAL_NURSES_FTE,TOTAL_DPC_HC,TOTAL_DPC_FTE,TOTAL_ADMIN_HC,TOTAL_ADMIN_FTE
0,A81001,4,4.133333333,1,0.52,1,0.426666667,10,7.293333
1,A81002,14,11.52,8,6.986666667,5,4.493333333,36,27.826667
2,A81004,3,2.56,8,7.333333333,4,2.873333333,19,15.186667
3,A81005,9,5.973333333,5,2.16,1,0.613333333,19,12.950133
4,A81006,13,9.72,6,5,6,4.36,24,18.893333
...,...,...,...,...,...,...,...,...,...
6424,Y07059,7,4.457333333,11,8.68,7,5.426666667,22,15.666667
6425,Y07060,0,0,0,0,0,0,0,0.0
6426,Y07274,0,0,0,0,0,0,0,0.0
6427,Y07275,1,1.066666667,0,0,1,0.213333333,3,2.52


#### TODO Transform totals and merge with df